In [1]:
# stage_procedimentos
# Processa a camada stage das bases de procedimentos e seus derivados

In [2]:
%run ../spark-default.py

In [24]:
from delta.tables import DeltaTable
import builtins as b

In [5]:
df_procedimento = spark.read.parquet("s3a://datalake/raw/proc_procedimento")
df_descricao = spark.read.parquet("s3a://datalake/raw/proc_descricao")

df_grupo = spark.read.parquet("s3a://datalake/raw/proc_grupo")
df_sub_grupo = spark.read.parquet("s3a://datalake/raw/proc_sub_grupo")
df_forma_organizacao = spark.read.parquet("s3a://datalake/raw/proc_forma_organizacao")

df_rl_procedimento_cid = spark.read.parquet("s3a://datalake/raw/proc_rl_procedimento_cid")

In [19]:
df_procedimento.show(truncate=False)
df_procedimento.select("dt_competencia").distinct().show(truncate=False)

+---------------+------------------------------------------------------------------+---------------+-------+------------------+-------------------+---------+---------------+---------------+----------+----------+----------+----------------+----------+--------------------+--------------+
|co_procedimento|no_procedimento                                                   |tp_complexidade|tp_sexo|qt_maxima_execucao|qt_dias_permanencia|qt_pontos|vl_idade_minima|vl_idade_maxima|vl_sh     |vl_sa     |vl_sp     |co_financiamento|co_rubrica|qt_tempo_permanencia|dt_competencia|
+---------------+------------------------------------------------------------------+---------------+-------+------------------+-------------------+---------+---------------+---------------+----------+----------+----------+----------------+----------+--------------------+--------------+
|0101010010     |ATIVIDADE EDUCATIVA / ORIENTAÇÃO EM GRUPO NA ATENÇÃO PRIMÁRIA     |1              |N      |9999              |9999        

In [35]:
df_final = df_procedimento.alias("proc") \
            .join(
                df_descricao.alias("desc"),
                col("desc.co_procedimento") == col("proc.co_procedimento"),
                "left"
            ) \
            .withColumn("proc_co_procedimento", substring("proc.co_procedimento", 0, 2)) \
            .withColumn("proc_co_sub_grupo", substring("proc.co_procedimento", 2, 2)) \
            .withColumn("proc_co_forma_organizacao", substring("proc.co_procedimento", 4, 2)) \
            .join(
                df_grupo.alias("grup"),
                [
                    col("proc_co_procedimento") == col("grup.co_grupo")
                ],
                "left"
            ) \
            .join(
                df_sub_grupo.alias("subg"),
                [
                    col("proc_co_procedimento") == col("subg.co_grupo"),
                    col("proc_co_procedimento") == col("subg.co_sub_grupo")
                ],
                "left"
            ) \
            .join(
                df_forma_organizacao.alias("form"),
                [
                    col("proc_co_procedimento") == col("form.co_grupo"),
                    col("proc_co_procedimento") == col("form.co_sub_grupo"),
                    col("proc_co_procedimento") == col("form.co_forma_organizacao")
                ],
                "left"
            ) \
            .select(
                col("proc.co_procedimento"),
                col("proc.no_procedimento"),
                col("proc.tp_complexidade"),
                col("proc.tp_sexo"),
                col("proc.qt_maxima_execucao"),
                col("proc.qt_dias_permanencia"),
                col("proc.qt_pontos"),
                col("proc.vl_idade_minima"),
                col("proc.vl_idade_maxima"),
                col("proc.vl_sh"),
                col("proc.vl_sa"),
                col("proc.vl_sp"),
                col("proc.co_financiamento"),
                col("proc.co_rubrica"),
                col("proc.qt_tempo_permanencia"),
                col("desc.ds_procedimento"),
                col("grup.no_grupo"),
                col("subg.no_sub_grupo"),
                col("form.no_forma_organizacao")
            )

df_final.show(truncate=False)

+---------------+------------------------------------------------------------------+---------------+-------+------------------+-------------------+---------+---------------+---------------+----------+----------+----------+----------------+----------+--------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [36]:
df_final_nulls = df_final.select(count("*").alias("total"), *[count(when(col(c).isNull(), c)).alias(c) for c in df_final.columns]).first().asDict()

df_final_nulls_total = df_final_nulls["total"]

df_final_nulls = spark.createDataFrame(
    [{"col": k, "count": v, "percentual": b.round((v/df_final_nulls_total) * 100, 2)} for k,v in df_final_nulls.items()]
) #.orderBy(col("count").desc())

df_final_nulls.show(200, truncate=False)

+--------------------+-----+----------+
|col                 |count|percentual|
+--------------------+-----+----------+
|total               |4865 |100.0     |
|co_procedimento     |0    |0.0       |
|no_procedimento     |0    |0.0       |
|tp_complexidade     |0    |0.0       |
|tp_sexo             |0    |0.0       |
|qt_maxima_execucao  |0    |0.0       |
|qt_dias_permanencia |0    |0.0       |
|qt_pontos           |0    |0.0       |
|vl_idade_minima     |0    |0.0       |
|vl_idade_maxima     |0    |0.0       |
|vl_sh               |0    |0.0       |
|vl_sa               |0    |0.0       |
|vl_sp               |0    |0.0       |
|co_financiamento    |0    |0.0       |
|co_rubrica          |0    |0.0       |
|qt_tempo_permanencia|0    |0.0       |
|ds_procedimento     |805  |16.55     |
|no_grupo            |0    |0.0       |
|no_sub_grupo        |1007 |20.7      |
|no_forma_organizacao|2845 |58.48     |
+--------------------+-----+----------+



In [37]:
DeltaTable.createIfNotExists(spark) \
  .tableName("stage.procedimentos") \
  .addColumns(df_final.schema) \
  .execute()

In [38]:
df_final.write.insertInto("stage.procedimentos")

In [39]:
spark.stop()